In [1]:
from transformers import LayoutLMv3FeatureExtractor
from transformers import LayoutLMv3Tokenizer
from PIL import Image
import os 
from utils import utils
import torch
import sys
# 一個上の階層をpathに追加
sys.path.append('../')
from model import mlm_layoutLMv3 
import matplotlib.pyplot as plt
import pandas as pd
import pickle

/home/is/hikaru-si/.pyenv/versions/3.8.6/envs/exp_004/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_path = "../../datasets/pdfs/processing/encode_train.pkl"
with open(file_path, 'rb') as f:
    dataset = pickle.load(f)

In [3]:
len(dataset)

233192

In [8]:
torch.tensor(dataset[0]["pixel_values"]).shape

torch.Size([3, 224, 224])

In [1]:
import simplejson as json
file_path = "../../datasets/pdfs/processing/encoding_train.json"
# file_path = "./en.json"

In [2]:
# df = pd.read_json(file_path)

In [2]:
import pickle

In [3]:
# with open(file_path, "r") as f:
#     data = json.load(f)

MemoryError: 

In [ ]:
len(data)

NameError: name 'data' is not defined

In [3]:
# json_open = open("../../datasets/pdfs/processing/encoding_train.json", 'r')
# dataset = json.load("../../datasets/pdfs/processing/encoding_train.json")

AttributeError: 'str' object has no attribute 'read'

In [3]:
file_path = "../../datasets/pdfs/images/ex"
image_names = os.listdir(file_path)
len(image_names)

100

In [4]:
file_names = []
for n in image_names:
    name = os.path.splitext(n)[0]
    file_names.append(name)
image_names.clear()

In [5]:
file_path = "../../datasets/pdfs/train/"

In [6]:
words, bboxes = utils.extraction_text_from_pdf(file_path, file_names)

In [6]:
# tokenizer = LayoutLMv3Tokenizer("../code/layoutlmv3-publaynet/vocab.json", "../code/layoutlmv3-publaynet/merges.txt")
# enc = tokenizer(text=words, boxes = bboxes, add_special_tokens=False)


In [7]:
# tokenizer = LayoutLMv3Tokenizer.from_pretrained("microsoft/layoutlmv3-base")
# enc = tokenizer(text=words, boxes = bboxes, add_special_tokens=False)

In [7]:
tokenizer = LayoutLMv3Tokenizer("../model/tokenizer_vocab/vocab.json", "../model/tokenizer_vocab/merges.txt")
enc = tokenizer(text=words, boxes = bboxes, add_special_tokens=False)

In [8]:
pixel_values = []
feature_extractor = LayoutLMv3FeatureExtractor(apply_ocr=False)
for file_name in file_names[:100]:
    image = Image.open(f"../../datasets/pdfs/images/train/{file_name}.png")
    pixel_value = feature_extractor(image)["pixel_values"]
    pixel_values.append(pixel_value)


len(pixel_values)

100

In [9]:
pixel_values[0][0].shape

(3, 224, 224)

## vocab

In [10]:
tokenizer = LayoutLMv3Tokenizer("../model/tokenizer_vocab/vocab.json", "../model/tokenizer_vocab/merges.txt")
ids = range(tokenizer.vocab_size)
vocab = tokenizer.convert_ids_to_tokens(ids)
vocab.index("benz")

9030

## 512の長さに分割

In [11]:
tokens, bboxes, doc_ids = utils.subset_tokens_from_document_light(enc.input_ids, enc.bbox, vocab, max_len=512)

In [13]:
len(tokens), len(doc_ids), doc_ids[:10]

(223, 223, [0, 0, 1, 1, 2, 3, 3, 4, 4, 4])

In [14]:
pixel_values[0]

[array([[[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],
 
        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]],
 
        [[1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         ...,
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 1., 1.]]], dtype=float32)]

In [12]:
# encoding = {"input_ids": o_tokens, "attention_mask": t[2], "bbox": t[1], "pixel_values":t[3], }
dataset = []
for i in range(len(tokens)):
    dataset.append({"input_ids": tokens[i],"bbox":bboxes[i], "pixel_values": pixel_values[doc_ids[i]][0]})

In [13]:
import json
from json import JSONEncoder
import numpy

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, numpy.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

In [18]:
#  file_path = "./en.json"
#  with open(file_path, 'w') as f:
#     json.dump(dataset, f, indent=4, cls=NumpyArrayEncoder)

In [ ]:
file_path = "./en.pkl"
with open(file_path, "wb") as f:
  pickle.dump(dataset,f, protocol=5)

In [36]:
def json_tensor_obj_hook(dct):
    """Decodes a previously encoded numpy ndarray with proper shape and dtype.

    :param dct: (dict) json encoded ndarray
    :return: (ndarray) if input was an encoded ndarray
    """
    print(dct)
    if isinstance(dct, list) :
        return torch.tensor(dct)
    return dct

In [37]:
with open(file_path, "r") as f:
    data = json.load(f, object_hook=json_tensor_obj_hook)


In [38]:
len(data)

223

In [39]:
data[0].keys()

dict_keys(['input_ids', 'bbox', 'pixel_values'])

In [40]:
data[0]["pixel_values"]

[[[1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,
   1.0,


In [21]:
data_loader = torch.utils.data.DataLoader(dataset, batch_size= 16, shuffle=True)